In [ ]:
!pip install ultralytics

In [ ]:
!pip install opencv-python

In [ ]:
import os
import numpy as np
import cv2
from ultralytics import YOLO
from collections import defaultdict

In [ ]:
model_BN_path = "/kaggle/input/models/yolo11l_3classes1080.pt"
model_MN_path = "/kaggle/input/models/yolo11s_1classe1080.pt"

model_BN = YOLO(model_BN_path)
model_MN = YOLO(model_MN_path)

In [ ]:
model_BN.predict(source="/kaggle/input/detectionbnmn-3classes/images/test", save=False, show_conf=False, show_labels=False, save_txt=True)
model_MN.predict(source="/kaggle/input/detectionbnmn-3classes/images/test", save=False, show_conf=False, show_labels=False, save_txt=True)

### Remoção da classe

In [ ]:
input_dir = '/kaggle/input/predicts-ensemble/predictBN11l/kaggle/working/runs/detect/predict/labels'
output_dir = '/kaggle/working/rmv_predict_11l'

class_id_to_remove = '2'

# Cria o diretório de saída, caso ele não exista
os.makedirs(output_dir, exist_ok=True)

# Loop pelos arquivos de anotação no diretório de entrada
for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        # Ler o arquivo e filtrar as linhas
        with open(input_path, 'r') as file:
            lines = file.readlines()

        # Manter apenas as linhas que não têm o class_id que queremos remover
        new_lines = [line for line in lines if not line.startswith(class_id_to_remove)]

        # Escrever o novo arquivo no diretório de saída
        with open(output_path, 'w') as file:
            file.writelines(new_lines)

print("Anotações da classe MN removidas e salvas no novo diretório com sucesso.")


### Junção das classes

In [ ]:
input_dir1 = '/kaggle/working/rmv_predict_11l'
input_dir2 = '/kaggle/input/predicts-ensemble/predictMN11s/kaggle/working/runs/detect/predict2/labels'

output_dir = '/kaggle/working/final_predict_ensemble'

# Cria o diretório de saída, caso ele não exista
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir1):
    if filename.endswith('.txt'):
        input_path1 = os.path.join(input_dir1, filename)
        input_path2 = os.path.join(input_dir2, filename)
        output_path = os.path.join(output_dir, filename)

        with open(input_path1, 'r') as file1:
            dir1 = file1.readlines()

        if os.path.exists(input_path2):
            with open(input_path2, 'r') as file2:
                dir2 = file2.readlines()
        else: 
            dir2 = []

        final_dir = dir1 + dir2

        with open(output_path, 'w') as f_out:
            f_out.writelines(final_dir)

print("Processo concluído")

### Cálculo das métricas

In [ ]:
import os
import numpy as np
from collections import defaultdict

def load_yolo_labels(txt_path):
    """Carrega as anotações YOLO de um arquivo TXT."""
    labels = []
    with open(txt_path, 'r') as f:
        for line in f.readlines():
            values = list(map(float, line.strip().split()))
            labels.append(values)  # Formato: [classe, x_c, y_c, w, h]
    return labels

def yolo_to_bbox(label, img_width=1, img_height=1):
    """Converte as coordenadas YOLO para coordenadas absolutas."""
    classe, x_c, y_c, w, h = label[:5]
    x_min = (x_c - w / 2) * img_width
    y_min = (y_c - h / 2) * img_height
    x_max = (x_c + w / 2) * img_width
    y_max = (y_c + h / 2) * img_height
    return [int(classe), x_min, y_min, x_max, y_max]

def calculate_iou(box1, box2):
    """Calcula o IoU entre duas bounding boxes."""
    x1_min, y1_min, x1_max, y1_max = box1[1:]
    x2_min, y2_min, x2_max, y2_max = box2[1:]
    
    xi1 = max(x1_min, x2_min)
    yi1 = max(y1_min, y2_min)
    xi2 = min(x1_max, x2_max)
    yi2 = min(y1_max, y2_max)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

def compute_ap(recalls, precisions):
    """
    Calcula o AP (Average Precision) usando interpolação da curva de precisão-recall.
    """
    recalls = np.array(recalls)
    precisions = np.array(precisions)

    # Adiciona (0,1) e (1,0) para garantir a interpolação correta
    recalls = np.concatenate(([0.0], recalls, [1.0]))
    precisions = np.concatenate(([0.0], precisions, [0.0]))

    # Garante que a precisão seja monótona não crescente
    for i in range(len(precisions) - 1, 0, -1):
        precisions[i - 1] = max(precisions[i - 1], precisions[i])

    # Encontra os pontos onde o recall muda
    indices = np.where(np.diff(recalls) > 0)[0]

    # Calcula a área sob a curva de precisão-recall (AP)
    ap = np.sum((recalls[indices + 1] - recalls[indices]) * precisions[indices + 1])

    return ap

def evaluate_detections(gt_dir, pred_dir, iou_thresholds=[0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]):
    """Avalia as predições YOLO em relação ao ground truth."""
    gt_files = {f[:-4]: os.path.join(gt_dir, f) for f in os.listdir(gt_dir) if f.endswith('.txt')}
    pred_files = {f[:-4]: os.path.join(pred_dir, f) for f in os.listdir(pred_dir) if f.endswith('.txt')}
    
    TP, FP, FN = defaultdict(lambda: defaultdict(int)), defaultdict(lambda: defaultdict(int)), defaultdict(lambda: defaultdict(int))
    
    for file_id in gt_files.keys():
        gt_labels = [yolo_to_bbox(lbl) for lbl in load_yolo_labels(gt_files[file_id])]

        if file_id in pred_files:
            pred_labels = [yolo_to_bbox(lbl) for lbl in load_yolo_labels(pred_files[file_id])]

        else: 
            pred_labels = []
        
        for iou_threshold in iou_thresholds:
            matched = set()
            
            for pred in pred_labels:
                best_iou = 0
                best_match = None
                for idx, gt in enumerate(gt_labels):
                    if gt[0] == pred[0]:  # Mesma classe
                        iou = calculate_iou(pred, gt)
                        # iou = get_iou(gt, pred)
                        if iou > best_iou:
                            best_iou = iou
                            best_match = idx
                
                if best_iou >= iou_threshold and best_match is not None and best_match not in matched:
                    TP[iou_threshold][pred[0]] += 1
                    matched.add(best_match)
                else:
                    FP[iou_threshold][pred[0]] += 1
            
            for idx, gt in enumerate(gt_labels):
                if idx not in matched:
                    FN[iou_threshold][gt[0]] += 1
    
    metrics = {}
    for iou_threshold in iou_thresholds:
        precisions, recalls, aps = {}, {}, {}
        for cls in [0, 1, 2]:
            precisions[cls] = TP[iou_threshold][cls] / (TP[iou_threshold][cls] + FP[iou_threshold][cls]) if (TP[iou_threshold][cls] + FP[iou_threshold][cls]) > 0 else 0
            recalls[cls] = TP[iou_threshold][cls] / (TP[iou_threshold][cls] + FN[iou_threshold][cls]) if (TP[iou_threshold][cls] + FN[iou_threshold][cls]) > 0 else 0
            # aps[cls] = precisions[cls] * recalls[cls]  # Aproximação do AP
        metrics[iou_threshold] = {"precisions": precisions, "recalls": recalls, "aps": aps}

    aps = {cls: compute_ap(
        [metrics[iou]['recalls'][cls] for iou in iou_thresholds], 
        [metrics[iou]['precisions'][cls] for iou in iou_thresholds])
        for cls in [0, 1, 2]}  # Alterar para `set(TP[iou_threshold].keys())` se quiser todas as classes dinâmicas
    
    # Adiciona os APs ao dicionário de métricas
    for iou_threshold in iou_thresholds:
        metrics[iou_threshold]["aps"] = aps
    
    return metrics

gt_dir = '/kaggle/input/detectionbnmn-3classes/labels/test'
pred_dir = '/kaggle/input/predicts-ensemble/predict_ensemble/kaggle/working/final_predict_ensemble'
metrics = evaluate_detections(gt_dir, pred_dir, iou_thresholds=[0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95])

print("Classe | Precisão | Recall | mAP50 | mAP50-95")
print("--------------------------------------------------")
for cls in [0, 1, 2]:
    prec50 = metrics[0.5]['precisions'][cls]
    rec50 = metrics[0.5]['recalls'][cls]
    ap50 = metrics[0.5]['aps'][cls]
    ap5095 = (metrics[0.5]['aps'][cls] + metrics[0.55]['aps'][cls] + metrics[0.60]['aps'][cls] + metrics[0.65]['aps'][cls] + metrics[0.70]['aps'][cls] + metrics[0.75]['aps'][cls] + metrics[0.80]['aps'][cls] + metrics[0.85]['aps'][cls] + metrics[0.90]['aps'][cls] + metrics[0.95]['aps'][cls]) / 10  # Aproximação para mAP50-95
    print(f"{cls:^6} | {prec50:.4f} | {rec50:.4f} | {ap50:.4f} | {ap5095:.4f}")
